In [3]:
import pyprind
import pandas as pd
import os
import sys

basepath = '/media/DOWNLOADS/aclImdb'
labels = {'pos': 1, 'neg': 0}

# Count total files dynamically
n_files = 0
for s in ('test', 'train'):
    for l in ('pos', 'neg'):
        path = os.path.join(basepath, s, l)
        n_files += len(os.listdir(path))

pbar = pyprind.ProgBar(n_files, stream=sys.stdout)

# Use lists for efficiency
data = []

for s in ('test', 'train'):
    for l in ('pos', 'neg'):
        path = os.path.join(basepath, s, l)
        for file in sorted(os.listdir(path)):
            with open(os.path.join(path, file), 'r', encoding='utf-8') as infile:
                txt = infile.read()
            data.append([txt, labels[l]])
            pbar.update()

# Create DataFrame once at the end
df = pd.DataFrame(data, columns=['review', 'sentiment'])

print('\nDataFrame created successfully!')
print(df.head())


0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00

DataFrame created successfully!
                                              review  sentiment
0  I went and saw this movie last night after bei...          1
1  Actor turned director Bill Paxton follows up h...          1
2  As a recreational golfer with some knowledge o...          1
3  I saw this film in a sneak preview, and it is ...          1
4  Bill Paxton has taken the true story of the 19...          1


In [5]:
import numpy as np
np.random.seed(0)
df=df.reindex(np.random.permutation(df.index))
df.to_csv('movie_data.csv',index=False,encoding='utf-8')
df=pd.read_csv('movie_data.csv',encoding='utf-8')
df=df.rename(columns={"0":"review","1":"sentiment"})
df.head(3)

,review,sentiment
0,"In 1974, the teenager Martha Moxley (Maggie Gr...",1
1,OK... so... I really like Kris Kristofferson a...,0
2,"***SPOILER*** Do not read this, if you think a...",0


In [7]:
df.shape

(50000, 2)

In [10]:
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
count=CountVectorizer()
docs = np.array(['The sun is shining',
'The weather is sweet',
'The sun is shining, the weather is sweet,'
'and one and one is two'])
bag=count.fit_transform(docs)
print(count.vocabulary_)
print(bag.toarray())

{'the': 6, 'sun': 4, 'is': 1, 'shining': 3, 'weather': 8, 'sweet': 5, 'and': 0, 'one': 2, 'two': 7}
[[0 1 0 1 1 0 1 0 0]
 [0 1 0 0 0 1 1 0 1]
 [2 3 2 1 1 1 2 1 1]]


In [11]:
from sklearn.feature_extraction.text import TfidfTransformer
tfidf=TfidfTransformer(use_idf=True,norm='l2',smooth_idf=True)
np.set_printoptions(precision=2)
print(tfidf.fit_transform(count.fit_transform(docs)).toarray())

[[0.   0.43 0.   0.56 0.56 0.   0.43 0.   0.  ]
 [0.   0.43 0.   0.   0.   0.56 0.43 0.   0.56]
 [0.5  0.45 0.5  0.19 0.19 0.19 0.3  0.25 0.19]]


In [16]:
import re 
def preprocessor(text):
    text=re.sub(r'<[^>]*>','',text)
    emoticons = re.findall(r'(?::|;|=)(?:-)?(?:\)|\(|D|P)',text)
    text = (re.sub(r'[\W]+', ' ', text.lower()) +
    ' '.join(emoticons).replace('-', ''))
    return text

In [17]:
preprocessor(df.loc[0,'review'][-50:])
preprocessor("</a>This :) is :( a test :-)!")

'this is a test :) :( :)'

In [18]:
def tokenizer(text):
    return text.split()
tokenizer('runners like running and thus they run')

['runners', 'like', 'running', 'and', 'thus', 'they', 'run']

In [21]:
from nltk.stem.porter import PorterStemmer
porter=PorterStemmer()
def tokenizer_porter(text):
    return [porter.stem(word) for word in text.split()]
tokenizer_porter('runners like running and thus they run')

['runner', 'like', 'run', 'and', 'thu', 'they', 'run']

In [22]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /home/icy-
[nltk_data]     bear/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [23]:
from nltk.corpus import stopwords
stop=stopwords.words('english')
[w for w in tokenizer_porter('a runner likes'
' running and runs a lot')
if w not in stop]

['runner', 'like', 'run', 'run', 'lot']

In [28]:
X_train = df.loc[:25000, 'review'].values
y_train = df.loc[:25000, 'sentiment'].values
X_test = df.loc[25000:, 'review'].values
y_test = df.loc[25000:, 'sentiment'].values


In [29]:
X_train.shape,y_train.shape,X_test.shape,y_test.shape

((25001,), (25001,), (25000,), (25000,))

In [35]:
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf=TfidfVectorizer(strip_accents=None,lowercase=False,preprocessor=None)
small_param_grid = [
{
'vect__ngram_range': [(1, 1)],
'vect__stop_words': [None],
'vect__tokenizer': [tokenizer, tokenizer_porter],
'clf__penalty': ['l2'],
'clf__C': [1.0, 10.0]

},
{

'vect__ngram_range': [(1, 1)],
'vect__stop_words': [stop, None],
'vect__tokenizer': [tokenizer],
'vect__use_idf':[False],
'vect__norm':[None],
'clf__penalty': ['l2'],
'clf__C': [1.0, 10.0]


},
]
lr_tfidf=Pipeline([('vect',tfidf),('clf',LogisticRegression(solver='liblinear'))
                   ])
gs_lr_tfidf=GridSearchCV(lr_tfidf,small_param_grid,scoring='accuracy',cv=5,verbose=2,n_jobs=-1)
gs_lr_tfidf.fit(X_train,y_train)

Fitting 5 folds for each of 8 candidates, totalling 40 fits


/media/WORK/CODING/rashcka_ml/venv/lib/python3.12/site-packages/sklearn/feature_extraction/text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/media/WORK/CODING/rashcka_ml/venv/lib/python3.12/site-packages/sklearn/feature_extraction/text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/media/WORK/CODING/rashcka_ml/venv/lib/python3.12/site-packages/sklearn/feature_extraction/text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/media/WORK/CODING/rashcka_ml/venv/lib/python3.12/site-packages/sklearn/feature_extraction/text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/media/WORK/CODING/rashcka_ml/venv/lib/python3.12/site-packages/sklearn/feature_extraction/text.py:517: UserWarning: The parameter 'token_pattern' will 

[CV] END clf__C=1.0, clf__penalty=l2, vect__ngram_range=(1, 1), vect__stop_words=None, vect__tokenizer=<function tokenizer at 0x747838e33240>; total time=   4.3s


/media/WORK/CODING/rashcka_ml/venv/lib/python3.12/site-packages/sklearn/feature_extraction/text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/media/WORK/CODING/rashcka_ml/venv/lib/python3.12/site-packages/sklearn/feature_extraction/text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


[CV] END clf__C=1.0, clf__penalty=l2, vect__ngram_range=(1, 1), vect__stop_words=None, vect__tokenizer=<function tokenizer at 0x7be822a33240>; total time=   4.5s
[CV] END clf__C=1.0, clf__penalty=l2, vect__ngram_range=(1, 1), vect__stop_words=None, vect__tokenizer=<function tokenizer at 0x75df08a4f240>; total time=   4.6s


...

[CV] END clf__C=1.0, clf__penalty=l2, vect__ngram_range=(1, 1), vect__norm=None, vect__stop_words=['a', 'about', 'above', 'after', 'again', 'against', 'ain', 'all', 'am', 'an', 'and', 'any', 'are', 'aren', "aren't", 'as', 'at', 'be', 'because', 'been', 'before', 'being', 'below', 'between', 'both', 'but', 'by', 'can', 'couldn', "couldn't", 'd', 'did', 'didn', "didn't", 'do', 'does', 'doesn', "doesn't", 'doing', 'don', "don't", 'down', 'during', 'each', 'few', 'for', 'from', 'further', 'had', 'hadn', "hadn't", 'has', 'hasn', "hasn't", 'have', 'haven', "haven't", 'having', 'he', "he'd", "he'll", 'her', 'here', 'hers', 'herself', "he's", 'him', 'himself', 'his', 'how', 'i', "i'd", 'if', "i'll", "i'm", 'in', 'into', 'is', 'isn', "isn't", 'it', "it'd", "it'll", "it's", 'its', 'itself', "i've", 'just', 'll', 'm', 'ma', 'me', 'mightn', "mightn't", 'more', 'most', 'mustn', "mustn't", 'my', 'myself', 'needn', "needn't", 'no', 'nor', 'not', 'now', 'o', 'of', 'off', 'on', 'once', 'only', 'or', 'o

/media/WORK/CODING/rashcka_ml/venv/lib/python3.12/site-packages/sklearn/feature_extraction/text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


[CV] END clf__C=1.0, clf__penalty=l2, vect__ngram_range=(1, 1), vect__norm=None, vect__stop_words=['a', 'about', 'above', 'after', 'again', 'against', 'ain', 'all', 'am', 'an', 'and', 'any', 'are', 'aren', "aren't", 'as', 'at', 'be', 'because', 'been', 'before', 'being', 'below', 'between', 'both', 'but', 'by', 'can', 'couldn', "couldn't", 'd', 'did', 'didn', "didn't", 'do', 'does', 'doesn', "doesn't", 'doing', 'don', "don't", 'down', 'during', 'each', 'few', 'for', 'from', 'further', 'had', 'hadn', "hadn't", 'has', 'hasn', "hasn't", 'have', 'haven', "haven't", 'having', 'he', "he'd", "he'll", 'her', 'here', 'hers', 'herself', "he's", 'him', 'himself', 'his', 'how', 'i', "i'd", 'if', "i'll", "i'm", 'in', 'into', 'is', 'isn', "isn't", 'it', "it'd", "it'll", "it's", 'its', 'itself', "i've", 'just', 'll', 'm', 'ma', 'me', 'mightn', "mightn't", 'more', 'most', 'mustn', "mustn't", 'my', 'myself', 'needn', "needn't", 'no', 'nor', 'not', 'now', 'o', 'of', 'off', 'on', 'once', 'only', 'or', 'o

/media/WORK/CODING/rashcka_ml/venv/lib/python3.12/site-packages/sklearn/feature_extraction/text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


,estimator,Pipeline(step...liblinear'))])
,param_grid,"[{'clf__C': [1.0, 10.0], 'clf__penalty': ['l2'], 'vect__ngram_range': [(1, ...)], 'vect__stop_words': [None], ...}, {'clf__C': [1.0, 10.0], 'clf__penalty': ['l2'], 'vect__ngram_range': [(1, ...)], 'vect__norm': [None], ...}]"
,scoring,'accuracy'
,n_jobs,-1
,refit,True
,cv,5
,verbose,2
,pre_dispatch,'2*n_jobs'
,error_score,nan
,return_train_score,False
,input,'content'


In [37]:
print(f'Best parameter set: {gs_lr_tfidf.best_params_}')
print(f'CV Accuracy: {gs_lr_tfidf.best_score_:.3f}')

Best parameter set: {'clf__C': 10.0, 'clf__penalty': 'l2', 'vect__ngram_range': (1, 1), 'vect__stop_words': None, 'vect__tokenizer': <function tokenizer at 0x7c5d2a62a2a0>}
CV Accuracy: 0.887


In [38]:
clf=gs_lr_tfidf.best_estimator_
print(f'Test Accuracy: {clf.score(X_test, y_test):.3f}')

Test Accuracy: 0.893


In [41]:
import numpy as np
import re
from nltk.corpus import stopwords
stop=stopwords.words('english')
def tokenizer(text):
    text = re.sub(r'<[^>]*>', '', text)
    emoticons = re.findall(r'(?::|;|=)(?:-)?(?:\)|\(|D|P)',text)
    text = re.sub(r'[\W]+', ' ', text.lower()) \
+ ' '.join(emoticons).replace('-', '')
    tokenized = [w for w in text.split() if w not in stop]
    return tokenized

In [43]:
def stream_docs(path):
    with open(path,'r',encoding='utf-8') as csv:
        next(csv)
        for line in csv:
            text,label=line[:-3],int(line[-2])
            yield text,label

In [44]:
next(stream_docs(path='movie_data.csv'))

('"In 1974, the teenager Martha Moxley (Maggie Grace) moves to the high-class area of Belle Haven, Greenwich, Connecticut. On the Mischief Night, eve of Halloween, she was murdered in the backyard of her house and her murder remained unsolved. Twenty-two years later, the writer Mark Fuhrman (Christopher Meloni), who is a former LA detective that has fallen in disgrace for perjury in O.J. Simpson trial and moved to Idaho, decides to investigate the case with his partner Stephen Weeks (Andrew Mitchell) with the purpose of writing a book. The locals squirm and do not welcome them, but with the support of the retired detective Steve Carroll (Robert Forster) that was in charge of the investigation in the 70\'s, they discover the criminal and a net of power and money to cover the murder.<br /><br />""Murder in Greenwich"" is a good TV movie, with the true story of a murder of a fifteen years old girl that was committed by a wealthy teenager whose mother was a Kennedy. The powerful and rich f

In [61]:
def get_minibatch(doc_stream,size):
    docs,y=[],[]
    try:
        for _ in range(size):
            text,label=next(doc_stream)
            docs.append(text)
            y.append(label)
    except StopIteration:
        pass
    return docs,y        

In [62]:
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.linear_model import SGDClassifier
vect=HashingVectorizer(decode_error='ignore',n_features=2**21,preprocessor=None,tokenizer=tokenizer)
clf=SGDClassifier(loss='log_loss',random_state=1)
doc_stream=stream_docs(path='movie_data.csv')

In [63]:
import pyprind
pbar = pyprind.ProgBar(45, stream=None)
classes=np.array([0,1])
for _ in range(45):
    X_train,y_train=get_minibatch(doc_stream,size=1000)
    if not X_train:
        break
    X_train=vect.transform(X_train)
    clf.partial_fit(X_train,y_train,classes=classes)
    pbar.update()

In [64]:
X_test, y_test = get_minibatch(doc_stream, size=5000)
X_test = vect.transform(X_test)
print(f'Accuracy: {clf.score(X_test, y_test):.3f}')

Accuracy: 0.868


In [65]:
import pandas as pd
df=pd.read_csv('movie_data.csv',encoding='utf-8')

In [66]:
from sklearn.feature_extraction.text import CountVectorizer
count=CountVectorizer(stop_words='english',max_df=.1,max_features=5000)
X=count.fit_transform(df['review'].values)

In [68]:
from sklearn.decomposition import LatentDirichletAllocation
lda=LatentDirichletAllocation(n_components=10,random_state=123,learning_method='batch',n_jobs=-1)
X_topics=lda.fit_transform(X)

In [69]:
lda.components_.shape

(10, 5000)

In [70]:
n_top_words=5
feature_names=count.get_feature_names_out()
for topic_idx,topic in enumerate(lda.components_):
 print(f'Topic {(topic_idx + 1)}:')
 print(' '.join([feature_names[i]
for i in topic.argsort()\
[:-n_top_words - 1:-1]]))

Topic 1:
horror worst minutes script budget
Topic 2:
dvd music video watched remember
Topic 3:
war american series history documentary
Topic 4:
game killer murder thriller horror
Topic 5:
kids comedy episode series school
Topic 6:
family woman beautiful mother feel
Topic 7:
role performance comedy plays john
Topic 8:
action effects horror john star
Topic 9:
book version original read music
Topic 10:
action wife father police guy


In [71]:
horror = X_topics[:, 5].argsort()[::-1]
for iter_idx, movie_idx in enumerate(horror[:3]):
 print(f'\nHorror movie #{(iter_idx + 1)}:')
 print(df['review'][movie_idx][:300], '...')


Horror movie #1:
May and her husband go to visit their children and grandchildren. The visit is awkward because the grandchildren and "kids" don't really seem to know each other as one might expect. The warmth that should be there is missing. After dinner, May's husband says he doesn't feel well, blames it on his da ...

Horror movie #2:
"Come Undone" appears to elicit a lot of opinions among the contributors to this forum. Granted, it's a film that promises a take on gay life, as most viewers expect and somehow, it gets away from that promise into an introspective view at a young man's soul. The film has a way of staying with us ev ...

Horror movie #3:
The subject of children being terminally ill is difficult and saddening but 'The Cure' successfully portrays the idea that it doesn't have to be all doom and gloom and, if anything, children need to have hope and delight in their lives if they are to find peace before the end. It is also a film of r ...
